In [17]:
import os
import tensorflow as tf
from keras import models
from keras import layers
from keras import Sequential
from datetime import datetime
from PIL import Image
import numpy as np
import keras.backend as K
import matplotlib.pyplot as plt
import mlflow
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt
from keras.models import Model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score

In [18]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset directory
dataset_dir = "dataset_2_final"



In [47]:
# Set image and batch size
image_size = (300, 300)
batch_size = 4

# Create the ImageDataGenerator with data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Splitting the dataset into training and validation sets
)

# Load the training dataset using image_dataset_from_directory
train_ds = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',  # For integer labels
    shuffle=True,
    subset='training',    # Selecting the training set
    seed=1337
)

# Load the validation dataset using image_dataset_from_directory
val_ds = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',  # For integer labels
    shuffle=False,        # No need to shuffle the validation set
    subset='validation',  # Selecting the validation set
    seed=1337
)


Found 423 images belonging to 7 classes.
Found 102 images belonging to 7 classes.


In [48]:
# Check the number of classes in your dataset
num_classes = len(train_ds.class_indices)
print(num_classes)

7


EfficientNetB0: Image input size is (224, 224, 3)
EfficientNetB1: Image input size is (240, 240, 3)
EfficientNetB2: Image input size is (260, 260, 3)
EfficientNetB3: Image input size is (300, 300, 3)
EfficientNetB4: Image input size is (380, 380, 3)
EfficientNetB5: Image input size is (456, 456, 3)
EfficientNetB6: Image input size is (528, 528, 3)
EfficientNetB7: Image input size is (600, 600, 3)

In [49]:
# Initialize the EfficientNetB3 model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=image_size + (3,))
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [50]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [51]:
# Train the model
epochs = 20
steps_per_epoch = len(train_ds)
validation_steps = len(val_ds)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)


Epoch 1/20
106/106 [==============================] - 114s 765ms/step - loss: 1.8764 - accuracy: 0.2931 - val_loss: 2.5003 - val_accuracy: 0.1961
Epoch 2/20
106/106 [==============================] - ETA: 0s - loss: 1.5946 - accuracy: 0.4303

KeyboardInterrupt: 

In [52]:

# Create the ImageDataGenerator with data augmentation and cropping
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Splitting the dataset into training and validation sets
)


In [59]:

# Custom data generator to generate four cropped images per input image

def custom_data_generator(data_generator, steps_per_epoch):
    for i, (images, labels) in enumerate(data_generator):
        batch_images = []
        batch_labels = []
        for image in images:
            # Crop the image into four quadrants
            height, width = image.shape[0], image.shape[1]
            half_height, half_width = height // 2, width // 2
            quadrants = [
                image[:half_height, :half_width, :],
                image[:half_height, half_width:, :],
                image[half_height:, :half_width, :],
                image[half_height:, half_width:, :],
            ]
            batch_images.extend(quadrants)
            batch_labels.extend([labels[i] for i in range(4)])

        batch_images = tf.stack(batch_images)
        batch_labels = tf.stack(batch_labels)
        yield batch_images, batch_labels

        # Check if we have generated enough batches (steps_per_epoch)
        if i >= steps_per_epoch:
            break



In [54]:

# Load the training dataset using image_dataset_from_directory
train_ds = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',  # For integer labels
    shuffle=True,
    subset='training',    # Selecting the training set
    seed=1337
)

# Load the validation dataset using image_dataset_from_directory
val_ds = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',  # For integer labels
    shuffle=False,        # No need to shuffle the validation set
    subset='validation',  # Selecting the validation set
    seed=1337
)

# Check the number of classes in your dataset
num_classes = len(train_ds.class_indices)


Found 423 images belonging to 7 classes.
Found 102 images belonging to 7 classes.


In [56]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [57]:

# Initialize the EfficientNetB3 model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=image_size + (3,))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)





In [60]:
# Train the model using the custom data generator
epochs = 20
# Number of steps per epoch for training
steps_per_epoch = len(train_ds)

# Number of steps for validation
validation_steps = len(val_ds)

# Create the custom data generator with cropping
train_generator = custom_data_generator(train_ds, steps_per_epoch)


Epoch 1/20
105/106 [============================>.] - ETA: 0s - loss: 1.9403 - accuracy: 0.2256

UnknownError: 2 root error(s) found.
  (0) Unknown:  IndexError: index 3 is out of bounds for axis 0 with size 3
Traceback (most recent call last):

  File "c:\Users\diosdadj\Anaconda3\envs\ml\lib\site-packages\tensorflow\python\ops\script_ops.py", line 249, in __call__
    ret = func(*args)

  File "c:\Users\diosdadj\Anaconda3\envs\ml\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 645, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\diosdadj\Anaconda3\envs\ml\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 892, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "c:\Users\diosdadj\Anaconda3\envs\ml\lib\site-packages\keras\engine\data_adapter.py", line 822, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\diosdadj\AppData\Local\Temp\ipykernel_24952\3572417806.py", line 18, in custom_data_generator
    batch_labels.extend([labels[i] for i in range(4)])

  File "C:\Users\diosdad

In [ ]:

# Train the model using the custom data generator
model.fit(
    train_generator,
    validation_data=val_ds,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

